# Build Table Describing Perturbations Used in this Experiment

In [1]:
import os
import pandas as pd

In [2]:
cell_health_dir = os.path.join("data", "cell_health_guides")
cell_painting_dir = os.path.join("data", "metadata", "platemap")

## Load and Process Cell Health Data

In [3]:
cell_health_cols = [
    "gene_name",
    "pert_name",
    "Clone ID",
    "Pathway/Cellular Process",
]

cell_painting_cols = [
    "gene_name",
    "pert_name",
    "broad_sample",
]

cell_lines = ["A549", "ES2", "HCC44"]

In [4]:
cell_health_file = os.path.join(cell_health_dir, "HCI_BenchmarksV1-2_platemap.csv")

cell_health_platemap_df = pd.read_csv(cell_health_file, sep=",")
cell_health_platemap_df.loc[:, "gene_name"] = (
    cell_health_platemap_df.loc[:, "gene_name"].str.replace("_", "-")
)
cell_health_platemap_df.loc[:, "pert_name"] = (
    cell_health_platemap_df.loc[:, "pert_name"].str.replace("_", "-")
)
print(cell_health_platemap_df.shape)
cell_health_platemap_df.head(2)

(384, 6)


,WellRow,WellCol,well_position,gene_name,pert_name,Clone ID
0,A,1,A01,Empty,EMPTY,NaN
1,B,1,B01,Chr2,Chr2-1,NaN


In [5]:
cell_health_annotation_file = os.path.join(cell_health_dir, "HCI_benchmarks_sgRNA_annotation.xlsx")

cell_health_annotation_df = pd.read_excel(cell_health_annotation_file, sep=",")
cell_health_annotation_df.loc[:, "guide id"] = (
    cell_health_annotation_df.loc[:, "guide id"].str.replace("_", "-")
)
cell_health_annotation_df.loc[:, "gene symbol"] = (
    cell_health_annotation_df.loc[:, "gene symbol"].str.replace("_", "-")
)

cell_health_annotation_df.loc[
    cell_health_annotation_df.loc[:, "gene symbol"].isna(),
    "gene symbol"
] = (
    cell_health_annotation_df.loc[:, "guide id"]
)

print(cell_health_annotation_df.shape)
cell_health_annotation_df.head(2)

(132, 31)


,guide_seq,guide id,gene symbol,Pathway/Cellular Process,class,HCC44_Demeter,HCC44_CRISPR Global Z,HCC44_Copy Number,ES2_gZ_3.12,A549_gZ_3.12,...,# Off-Target Tier III Match Bin III Matches,# Off-Target Tier IV Match Bin III Matches,# Off-Target Tier I Match Bin IV Matches,# Off-Target Tier II Match Bin IV Matches,# Off-Target Tier III Match Bin IV Matches,# Off-Target Tier IV Match Bin IV Matches,RS2 Score,ATARIS_cscore,ATARIS_pval,ATARIS_qval
0,NaN,MYC-1,MYC,Known cancer gene,exp,-2.214323,NaN,50.658132,-0.2016,-0.9548,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,MYC-2,MYC,Known cancer gene,exp,-2.214323,NaN,50.658132,-0.2016,-0.9548,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
cell_health_df = (
    cell_health_platemap_df.merge(
        cell_health_annotation_df,
        left_on=["gene_name", "pert_name"],
        right_on=["gene symbol", "guide id"],
        how="outer"
    )
    .loc[:, cell_health_cols]
    .drop_duplicates()
    .sort_values(by="gene_name")
    .reset_index(drop=True)
    .assign(cell_health_data=True)
    .dropna(subset=["gene_name"])
)

print(cell_health_df.shape)
cell_health_df.head(2)

(127, 5)


,gene_name,pert_name,Clone ID,Pathway/Cellular Process,cell_health_data
0,AKT1,AKT1-1,BRDN0001054908,PIK3CA,True
1,AKT1,AKT1-2,BRDN0001055115,PIK3CA,True


## Load and Process Cell Painting Data

In [7]:
cp_platemap_file = os.path.join(cell_painting_dir, "DEPENDENCIES1_A549.csv")
cp_platemap_df = (
    pd.read_csv(cp_platemap_file, sep=',')
    .loc[:, cell_painting_cols]
    .drop_duplicates()
    .assign(cell_painting_data=True)
)

print(cp_platemap_df.shape)
cp_platemap_df.head()

(119, 4)


,gene_name,pert_name,broad_sample,cell_painting_data
0,EMPTY,EMPTY,NaN,True
1,MCL1,MCL1-5,NaN,True
2,AKT1,AKT1-1,BRDN0001054908,True
3,KRAS,KRAS-2B,NaN,True
4,AKT1,AKT1-2,BRDN0001055115,True


In [8]:
full_df = (
    cell_health_df.merge(
        cp_platemap_df,
        left_on=["gene_name", "pert_name", "Clone ID"],
        right_on=["gene_name", "pert_name", "broad_sample"],
        how="outer"
    )
    .rename(
        {
            "Pathway/Cellular Process": "cell_process"
        },
        axis="columns"
    )
    .loc[:,
         [
             "gene_name",
             "pert_name",
             "broad_sample",
             "cell_process",
             "cell_health_data",
             "cell_painting_data"
         ]
        ]
    .sort_values(by=["gene_name", "pert_name"])
)

full_df.loc[full_df.cell_health_data.isna(), "cell_health_data"] = False
full_df.loc[full_df.cell_painting_data.isna(), "cell_painting_data"] = False

print(full_df.shape)
full_df.head()

(127, 6)


,gene_name,pert_name,broad_sample,cell_process,cell_health_data,cell_painting_data
0,AKT1,AKT1-1,BRDN0001054908,PIK3CA,True,True
1,AKT1,AKT1-2,BRDN0001055115,PIK3CA,True,True
2,ARID1B,ARID1B-1,NaN,Chromatin Modifiers,True,True
3,ARID1B,ARID1B-2,NaN,Chromatin Modifiers,True,True
5,ATF4,ATF4-1,NaN,ER Stress/UPR,True,True


## Output Supplementary Table

In [9]:
output_file = os.path.join("tables", "supplementary_table_1_perturbation_details.tsv")
full_df.to_csv(output_file, sep='\t', index=False)